In [218]:
import pandas as pd 
import numpy as np

#from sklearn.pipeline import Pipeline
#from sklearn.base import BaseEstimator, TransformerMixin
import matplotlib.pyplot as plt
#import gensim
#import torch
#from sklearn.manifold import TSNE
import seaborn as sns
import pyLDAvis
import pyLDAvis.lda_model
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel

from gensim import corpora
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import re

import time

[nltk_data] Downloading package stopwords to /Users/bahia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Metrique d'évaluation utilisé
  
  
Perplexity : La perplexité est une mesure de la qualité du modèle. Une perplexité plus faible indique un meilleur modèle. Vous pouvez obtenir ce score en utilisant la méthode perplexity.  
  
Coherence Score : Le score de cohérence mesure la cohérence sémantique des sujets générés par le modèle LDA. Un score de cohérence plus élevé indique que les mots les plus importants d'un sujet sont plus sémantiquement similaires les uns aux autres. Le score de cohérence n'est pas directement disponible dans scikit-learn, mais peut être calculé en utilisant la bibliothèque Gensim.
  
### LDA.  
L'Analyse de la Répartition Latente de Dirichlet (LDA) est une technique de modélisation de sujets qui est largement utilisée en traitement du langage naturel. Elle permet de découvrir les sujets cachés dans un ensemble de documents en attribuant des distributions de probabilité à la fois aux documents (sur les sujets) et aux sujets (sur les mots), ce qui permet d'obtenir une représentation non supervisée et interprétable des données textuelles.



### Lda avec gensim

In [219]:
# Classe de prétraitement
class GensimPreprocessor:
    def __init__(self, stop_words=stopwords.words('english')):
        self.stop_words = stop_words
        
    def tokenize(self, text):
        return [word for word in word_tokenize(text.lower()) if word.isalpha()]
    
    def remove_stopwords(self, tokens):
        return [word for word in tokens if word not in self.stop_words]
    
    def preprocess(self, documents):
        preprocessed = []
        for document in documents:
            tokens = self.tokenize(document)
            tokens = self.remove_stopwords(tokens)
            preprocessed.append(tokens)
        return preprocessed


In [220]:
'''def visualize_topics(document_topics, dim_names, document_topics_max):
    # Appliquer t-SNE aux sujets
    tsne_model = TSNE(n_components=2, random_state=0)
    tsne_topics = tsne_model.fit_transform(document_topics)

    # Créer un DataFrame avec les résultats t-SNE et les sujets
    df_tsne = pd.DataFrame(data=tsne_topics, columns=dim_names)
    df_tsne['Topic'] = document_topics_max

    # Créer un graphique de dispersion des sujets avec Seaborn
    plt.figure(figsize=(5, 5))
    sns.scatterplot(x=dim_names[0], y=dim_names[1], hue="Topic", palette="deep", data=df_tsne)
    plt.show()'''

'def visualize_topics(document_topics, dim_names, document_topics_max):\n    # Appliquer t-SNE aux sujets\n    tsne_model = TSNE(n_components=2, random_state=0)\n    tsne_topics = tsne_model.fit_transform(document_topics)\n\n    # Créer un DataFrame avec les résultats t-SNE et les sujets\n    df_tsne = pd.DataFrame(data=tsne_topics, columns=dim_names)\n    df_tsne[\'Topic\'] = document_topics_max\n\n    # Créer un graphique de dispersion des sujets avec Seaborn\n    plt.figure(figsize=(5, 5))\n    sns.scatterplot(x=dim_names[0], y=dim_names[1], hue="Topic", palette="deep", data=df_tsne)\n    plt.show()'

In [221]:
'''def stocker_results(name = str,loglikhood_train=None, perplexity_train=None, loglikhood_test=None, perplexity_test=None, fit_time=None, transform_time=None):
    # Enregistrer les résultats dans un DataFrame
    results = pd.DataFrame({'Vectorizer': [name], 'Log Likelihood_train': [loglikhood_train], 'Perplexity_train': [perplexity_train], 
    'Log Likelihood Test': [loglikhood_test], 'Perplexity Test': [perplexity_test], 'Fit Time': [fit_time]})
    display(results)
    return results'''


"def stocker_results(name = str,loglikhood_train=None, perplexity_train=None, loglikhood_test=None, perplexity_test=None, fit_time=None, transform_time=None):\n    # Enregistrer les résultats dans un DataFrame\n    results = pd.DataFrame({'Vectorizer': [name], 'Log Likelihood_train': [loglikhood_train], 'Perplexity_train': [perplexity_train], \n    'Log Likelihood Test': [loglikhood_test], 'Perplexity Test': [perplexity_test], 'Fit Time': [fit_time]})\n    display(results)\n    return results"

In [222]:
data = pd.read_csv('Data/df_tags_filtered.csv')
data['combined'] = data['Title'] + " " + data['Body']

display(data[:1])

def words_tokenize(text):
  text = text.split()
  return text

#data['Body'] = data.Body.apply(lambda text: words_tokenize(text))
#data['Title'] = data.Title.apply(lambda text: words_tokenize(text))
data['Tags'] = data.Tags.apply(lambda text: words_tokenize(text))
display(data[:1])

,Title,Body,Tags,Body_word_count,combined
0,android jetpack navigation bottomnavigationvie...,android jetpack navigation bottomnavigationvie...,android android-architecture-components bottom...,131,android jetpack navigation bottomnavigationvie...


,Title,Body,Tags,Body_word_count,combined
0,android jetpack navigation bottomnavigationvie...,android jetpack navigation bottomnavigationvie...,"[android, android-architecture-components, bot...",131,android jetpack navigation bottomnavigationvie...


In [223]:
X = data['combined'][:100]
y = data['Tags'][:100]

df = pd.DataFrame(data)
'''def merge_and_keep_first_occurrence(row):
    merged = row['Title'] + row['Body']
    return list(dict.fromkeys(merged))
'''
display(X[:1])

X_train, X_test, y_train, y_test = train_test_split(X,y,
    test_size=0.2,
    random_state=42
)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


0    android jetpack navigation bottomnavigationvie...
Name: combined, dtype: object

X_train shape: (80,)
X_test shape: (20,)
y_train shape: (80,)
y_test shape: (20,)


## Test avec LDA Gensim

train set

In [225]:
#documents = X_fulltrain['Body']
documents =  X#_train
display(documents)
print(documents.shape)
preprocessor = GensimPreprocessor()
preprocessed_documents = preprocessor.preprocess(documents)
lda = GensimLDA(num_topics=6, passes=10)
lda.fit(preprocessed_documents)

coherence_score = lda.get_coherence(preprocessed_documents)
corpus = [lda.dictionary.doc2bow(text) for text in preprocessed_documents]
test_documents_topics = [lda.get_topics(bow) for bow in corpus]

print("coherence score =",coherence_score)

0     android jetpack navigation bottomnavigationvie...
1     jetpack compose button drawable achieve jetpac...
2     topappbar flash navigate compose navigation sc...
3     how create recycler view compose jetpack speci...
4     how navhostfragment integrate android navigati...
                            ...                        
95    android m camera intent permission bug app rea...
96    jetpack compose collapse toolbar cant find doc...
97    android get resourcesnotfoundexception abcicab...
98    how clear navigation stack navigate another fr...
99    entity framework core addmigration create migr...
Name: combined, Length: 100, dtype: object

(100,)


AttributeError: 'GensimLDA' object has no attribute 'get_topics'

Test set

In [ ]:
#documents = X_test
'''print(documents.shape)
#preprocessor = GensimPreprocessor()
preprocessed_test_documents = preprocessor.preprocess(X_test)
print(X_test.shape)
# Supposons que `dictionary` est votre dictionnaire Gensim créé pendant l'entraînement
test_corpus = [lda.dictionary.doc2bow(doc) for doc in preprocessed_test_documents]

# Supposons que `lda_model` est votre modèle LDA Gensim entraîné
test_documents_topics = [lda.model.get_document_topics(bow) for bow in test_corpus]'''
test_documents_topics = [lda.get_document_topics(bow) for bow in corpus]
most_probable_topics = [max(doc, key=lambda x: x[1])[0] for doc in test_documents_topics]
# Calcul du score de cohérence sur le set de test
#coherence_model_test = CoherenceModel(model=lda.model, texts=preprocessed_test_documents, dictionary=lda.dictionary, coherence='c_v')
coherence_score_test = lda.get_coherence(preprocessed_test_documents)
print(f"Score de cohérence sur le set de test: {coherence_score_test}")



Score de cohérence sur le set de test: nan


/Users/bahia/opt/anaconda3/lib/python3.9/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in scalar divide
  m_lr_i = np.log(numerator / denominator)
/Users/bahia/opt/anaconda3/lib/python3.9/site-packages/gensim/topic_coherence/indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in scalar divide
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


In [ ]:
test_documents_topics = lda.model.get_document_topics(test_corpus, minimum_probability=0.0)
documents_topics_count_vec = np.array([[prob for _, prob in doc_topics] for doc_topics in test_documents_topics])
document_topics_max = np.argmax(documents_topics_count_vec, axis=1)

In [ ]:
import pyLDAvis.gensim_models
import pyLDAvis
# Assurez-vous d'appeler pyLDAvis.enable_notebook() dans un environnement Jupyter pour l'interaction inline
pyLDAvis.enable_notebook()

# Préparation des données pour PyLDAVis
vis = pyLDAvis.gensim_models.prepare(lda.model, test_corpus, dictionary=lda.dictionary)

# Visualisation
pyLDAvis.display(vis)

In [ ]:
# obtain topic distributions for each document
topic_dist = pd.DataFrame(test_documents_topics)
for topic in topic_dist.columns:
    topic_dist[topic] = topic_dist[topic].apply(lambda x : x[1])

print('matrix of document/topic distribution shape:', topic_dist.shape)
display(topic_dist.head())


matrix of document/topic distribution shape: (13, 6)


,0,1,2,3,4,5
0,0.163807,0.181148,0.003108,0.091383,0.178526,0.382027
1,0.200293,0.013912,0.118389,0.140211,0.013961,0.513235
2,0.080581,0.125457,0.069569,0.181734,0.258566,0.284094
3,0.015186,0.015170,0.015192,0.015216,0.015158,0.924077
4,0.231940,0.479067,0.001148,0.087057,0.001156,0.199631


In [ ]:

from sklearn.preprocessing import MultiLabelBinarizer


mlb = MultiLabelBinarizer()

# Adapter MultiLabelBinarizer et transformer les étiquettes
y_binarized = mlb.fit_transform(y_train)
y_test_binarized = mlb.transform(y_test) 
display(y_test_binarized.shape)


(20, 166)

In [ ]:
'''topic_tag = np.matmul(topic_dist.T, y_binarized)
# normalizing 
topic_tag = topic_tag/np.sum(y_test_binarized,axis=0)'''

'topic_tag = np.matmul(topic_dist.T, y_binarized)\n# normalizing \ntopic_tag = topic_tag/np.sum(y_test_binarized,axis=0)'

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.multiclass import OneVsRestClassifier


time1 =time.time()
x_train, x_test, y_train_lr, y_test_lr = train_test_split(topic_dist, y_binarized, test_size=0.2, random_state=100)
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
print("y_train shape:", y_train_lr.shape)
print("y_test shape:", y_test_lr.shape)



# Adapter MultiLabelBinarizer et transformer les étiquettes


lr = OneVsRestClassifier(LogisticRegression()).fit(x_train, y_train_lr)
y_pred_lr = lr.predict_proba(x_test)
fit_time = np.round(time.time() - time1,1)
#score of prediction



ValueError: Found input variables with inconsistent numbers of samples: [13, 80]

In [ ]:
probabilities = y_pred_lr

print(probabilities.shape, y_test_lr.shape)
# Définir un seuil personnalisé pour sélectionner les tags
threshold = 0.15  # baisser le seuil pour inclure plus de tags

def select_top_n_tags(probabilities, threshold, top_n=5):
    # Créer une matrice de zéros de la même forme que probabilities
    final_tags = np.zeros(probabilities.shape)
    
    for i in range(probabilities.shape[0]):
        # Trouver les indices qui satisfont le seuil
        eligible_indices = np.where(probabilities[i] >= threshold)[0]
        
        # Trier ces indices en fonction de leur probabilité
        top_indices = eligible_indices[np.argsort(probabilities[i][eligible_indices])][::-1]
        
        # Sélectionner les top N indices
        top_n_indices = top_indices[:top_n]
        
        # Mettre à 1 les positions correspondant aux tags sélectionnés
        final_tags[i, top_n_indices] = 1
    
    return final_tags

# Utiliser la fonction pour obtenir les tags prédits

binary_predictions = select_top_n_tags(probabilities, threshold, top_n=7)

(3, 84) (3, 84)


In [ ]:
print("Shape of true values:", y_test_lr.shape)
print("Shape of predictions:", binary_predictions.shape)

Shape of true values: (3, 84)
Shape of predictions: (3, 84)


In [ ]:
from sklearn.metrics import jaccard_score


print("jaccord score:", jaccard_score(binary_predictions, y_test_lr, average='samples'))
print("Precision:", precision_score(binary_predictions, y_test_lr, average='samples'))
print("Recall:", recall_score(binary_predictions, y_test_lr, average='samples'))
print("F1 Score:", f1_score(binary_predictions, y_test_lr, average='samples'))


jaccord score: 0.18835978835978837
Precision: 0.6333333333333333
Recall: 0.23809523809523805
F1 Score: 0.3156565656565657


In [ ]:
# Afficher les labels des prédictions
predictions_labels =mlb.inverse_transform(y_test_lr)
true_labels = mlb.inverse_transform(binary_predictions)
print(predictions_labels)
print(true_labels)

[('android',), ('android', 'android-compose-button', 'android-jetpack-compose', 'android-jetpack-compose-button'), ('android', 'android-5.0-lollipop', 'android-6.0-marshmallow', 'floating-action-button', 'material-design')]
[('android', 'android-architecture-navigation', 'android-fragments', 'android-jetpack-compose', 'android-navigation', 'floating-action-button', 'kotlin'), ('android', 'android-architecture-navigation', 'android-fragments', 'android-jetpack-compose', 'android-navigation', 'floating-action-button', 'kotlin'), ('android', 'android-architecture-navigation', 'android-fragments', 'android-jetpack-compose', 'android-navigation', 'floating-action-button', 'kotlin')]


In [ ]:


# Créer un DataFrame à partir des labels de prédiction et des vrais labels
df = pd.DataFrame({
    'Predicted Labels': predictions_labels,
    'True Labels': true_labels
})

display(df)

,Predicted Labels,True Labels
0,"(android,)","(android, android-architecture-navigation, and..."
1,"(android, android-compose-button, android-jetp...","(android, android-architecture-navigation, and..."
2,"(android, android-5.0-lollipop, android-6.0-ma...","(android, android-architecture-navigation, and..."
